# Tratamiento de los datos missing

La diferencia entre los datos que se encuentran en muchos tutoriales y los datos del mundo real es que los datos del mundo real rara vez son limpios y homogéneos.
En concreto, **en muchos conjuntos de datos interesantes faltan algunos datos**.
Para complicar aún más las cosas, las diferentes fuentes de datos pueden indicar los datos que faltan de diferentes maneras.

En esta sección, discutiremos algunas consideraciones generales para los datos que faltan, discutiremos cómo Pandas elige representarlos, y demostraremos algunas herramientas incorporadas de Pandas para manejar los datos que faltan en Python.
**Aquí y a lo largo del libro, nos referiremos a los datos perdidos en general como valores *nulos*, *NaN*, o *NA***

## Compromisos en las convenciones sobre datos missing

Se han desarrollado varios esquemas para indicar la presencia de datos que faltan en una tabla o DataFrame.
Generalmente, giran en torno a una de dos estrategias: **utilizar una *máscara* que indica globalmente los valores que faltan, o elegir un *valor centinela* que indica una entrada que falta**.

En el enfoque de enmascaramiento, **la máscara** puede ser una matriz booleana completamente separada, o puede implicar la apropiación de un bit en la representación de datos para indicar localmente el estado nulo de un valor.

En el enfoque de centinela, el valor centinela **puede ser alguna convención específica de los datos, como indicar que falta un valor entero con -9999** o algún patrón de bits raro, o puede ser una convención más global, como indicar que falta un valor de coma flotante con **NaN (Not a Number)**, un valor especial que forma parte de la especificación IEEE de coma flotante.

Ninguno de estos enfoques está exento de inconvenientes: el uso de una matriz de máscaras separada requiere la asignación de una matriz booleana adicional, lo que añade sobrecarga tanto de almacenamiento como de cálculo. Un valor centinela reduce el rango de valores válidos que pueden representarse y puede requerir lógica adicional (a menudo no optimizada) en la aritmética de la CPU y la GPU. Los valores especiales comunes, como NaN, no están disponibles para todos los tipos de datos.

Como en la mayoría de los casos en los que no existe una opción universalmente óptima, los distintos lenguajes y sistemas utilizan convenciones diferentes.
**Por ejemplo, el lenguaje R utiliza patrones de bits reservados dentro de cada tipo de datos como valores centinela que indican que faltan datos**, mientras que el sistema SciDB utiliza un byte adicional adjunto a cada celda que indica un estado NA.

## Missing en Pandas

La forma en que Pandas **maneja los valores perdidos está limitada por su dependencia del paquete NumPy**, que no tiene una noción incorporada de valores NA para tipos de datos que no sean de coma flotante.

Pandas podría haber seguido el ejemplo de **R especificando patrones de bits para cada tipo de datos individual** para indicar nulidad, pero este enfoque resulta ser bastante difícil de manejar.
Mientras que R contiene cuatro tipos de datos básicos, **NumPy soporta *mucho* más que esto**: por ejemplo, mientras que R tiene un único tipo entero, NumPy soporta *catorce* tipos enteros básicos una vez que se tienen en cuenta las precisiones disponibles, la signatura y la endianidad de la codificación.
**Reservar un patrón de bits específico en todos los tipos NumPy disponibles llevaría a una cantidad inmanejable de sobrecarga** en operaciones especiales para varios tipos, probablemente incluso requiriendo una nueva bifurcación del paquete NumPy. Además, para los tipos de datos más pequeños (como los enteros de 8 bits), sacrificar un bit para utilizarlo como máscara reducirá significativamente el rango de valores que puede representar.

**NumPy tiene soporte para arrays enmascarados** - es decir, arrays que tienen un array booleano de máscara separado para marcar los datos como "buenos" o "malos".
Pandas podría haber derivado de esto, pero la sobrecarga en almacenamiento, cálculo y mantenimiento de código hace que sea una opción poco atractiva.

Con estas limitaciones en mente, **Pandas optó por utilizar centinelas para los datos que faltan**, y además optó por utilizar dos valores nulos ya existentes en Python: el valor especial de punto flotante **``NaN``, y el objeto Python ``None``**.
Esta elección tiene algunos efectos secundarios, como veremos, pero en la práctica acaba siendo un buen compromiso en la mayoría de los casos de interés.

### ``None``: Datos Pythonic que faltan

El primer valor centinela utilizado por Pandas es **``None``**, un objeto Python singleton que se utiliza a menudo para los datos que faltan en el código Python.
Debido a que es un objeto Python, ``None`` no se puede utilizar en cualquier array arbitrario de NumPy/Pandas, sino sólo en arrays con tipo de datos ``'object'`` (es decir, arrays de objetos Python):

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
vals1 = np.array([1, None, 3, 4])
vals1

Este ``dtype=object`` significa que la mejor representación de tipo común que NumPy puede inferir para los contenidos del array es que son objetos Python.
Aunque este tipo de array objeto es útil para algunos propósitos, cualquier operación sobre los datos se hará a nivel Python, con mucha más sobrecarga que las típicamente rápidas operaciones vistas para arrays con tipos nativos:

In [ ]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

El uso de objetos Python en un array también significa que si realizas agregaciones como ``sum()`` o ``min()`` a través de un array con un valor ``None``, generalmente obtendrás un error:

In [ ]:
vals1.sum()

Esto refleja el hecho de que la suma entre un entero y ``None`` es indefinida.

### ``NaN``: Missing numerical data

La otra representación de datos que faltan, ``NaN`` (acrónimo de *Not a Number*), es diferente; se trata de un valor especial de coma flotante reconocido por todos los sistemas que utilizan la representación estándar IEEE de coma flotante:

In [ ]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

Observa que NumPy eligió un tipo nativo de coma flotante para este array: esto significa que **a diferencia del array objeto de antes, este array soporta operaciones rápidas introducidas en código compilado.**
Debes tener en cuenta que ``NaN`` es un poco como un virus de datos - infecta cualquier otro objeto que toca.
Independientemente de la operación, el resultado de la aritmética con ``NaN`` será otro ``NaN``:

In [ ]:
1 + np.nan

In [ ]:
0 *  np.nan

In [ ]:
vals2

Tenga en cuenta que esto significa que los agregados sobre los valores están bien definidos (es decir, no dan lugar a error) pero no siempre son útiles:

In [ ]:
vals2.sum(), vals2.min(), vals2.max()

NumPy proporciona algunas agregaciones especiales que ignorarán estos valores perdidos:

In [ ]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

Tenga en cuenta que ``NaN`` es específicamente un valor de coma flotante; no existe un valor NaN equivalente para enteros, cadenas u otros tipos.

### NaN and None in Pandas

Tanto ``NaN`` como ``None`` tienen su lugar, y Pandas está construido para manejar los dos casi indistintamente, convirtiendo entre ellos cuando sea apropiado:

In [ ]:
pd.Series([1, np.nan, 2, None])

Para los tipos que no tienen un valor centinela disponible, Pandas realiza automáticamente una conversión de tipo cuando hay valores NA presentes.
Por ejemplo, **si establecemos un valor en un array de enteros a ``np.nan``, se convertirá automáticamente a un tipo de punto flotante para acomodar el NA**:

In [ ]:
x = pd.Series(range(2), dtype=int)
x

In [ ]:
x[0] = None
x

Observe que **además de convertir el array de enteros a coma flotante, Pandas convierte automáticamente el valor ``None`` a un valor ``NaN``.
(Tenga en cuenta que hay una propuesta para añadir un entero nativo NA a Pandas en el futuro; en el momento de escribir esto, no se ha incluido).

Mientras que este tipo de magia puede parecer un poco "hackish" en comparación con el enfoque más unificado de los valores NA en lenguajes de dominio específico como R, el enfoque centinela/casting de Pandas funciona bastante bien en la práctica y en mi experiencia sólo rara vez causa problemas.

La siguiente tabla lista las convenciones de upcasting en Pandas cuando se introducen valores NA:

| Typeclass     | Conversión al almacenar NA | NA Valor centinela     |
|--------------|-----------------------------|------------------------|
| ``floating`` | Sin cambios                   | ``np.nan``             |
| ``object``   | Sin cambios                   | ``None`` or ``np.nan`` |
| ``integer``  | Cambia a ``float64``         | ``np.nan``             |
| ``boolean``  | Cambia a ``object``          | ``None`` or ``np.nan`` |

Tenga en cuenta que en Pandas, los datos de cadena siempre se almacenan con un dtype ``object``.

## Operar con valores nulos

Como hemos visto, **Pandas trata ``None`` y ``NaN`` como esencialmente intercambiables para indicar valores perdidos o nulos.**
Para facilitar esta convención, existen varios métodos útiles para detectar, eliminar y reemplazar valores nulos en las estructuras de datos de Pandas.
Estos son:

- ``isnull()``: Genera una máscara booleana que indica valores perdidos
- ``notnull()``: Lo contrario de ``isnull()``
- ``dropna()``: Devuelve una versión filtrada de los datos
- ``fillna()``: Devuelve una copia de los datos con los valores perdidos rellenados o imputados

Concluiremos esta sección con una breve exploración y demostración de estas rutinas.

### Detección de valores nulos
Las estructuras de datos de Pandas tienen dos métodos útiles para detectar datos nulos: ``isnull()`` y ``notnull()``.
Cualquiera de ellos devolverá una máscara booleana sobre los datos. Por ejemplo:

In [ ]:
data = pd.Series([1, np.nan, 'hello', None])
data

In [ ]:
data.isnull()

In [ ]:
data.notnull()

Como se menciona en [Data Indexing and Selection](2_Data-Indexing-and-Selection.ipynb), las máscaras booleanas se pueden utilizar directamente como un índice ``Series`` o ``DataFrame``:

In [ ]:
data[data.notnull()]

In [ ]:
# data[~(data==1)]

In [ ]:
data[~data.isnull()]

Los métodos ``isnull()`` y ``notnull()`` producen resultados booleanos similares para los ``DataFrame``.

### Eliminación de valores nulos

Además del enmascaramiento utilizado anteriormente, existen los métodos ``dropna()`` (que elimina los valores nulos) y ``fillna()`` (que rellena los valores nulos).
(que elimina los valores nulos) y ``fillna()`` (que rellena los valores nulos). Para una ``Serie``,
el resultado es sencillo:

In [ ]:
data.dropna()
# data.dropna(inplace=True)
# data = data.dropna()

In [ ]:
data

Para un ``DataFrame``, hay más opciones.

Considera el siguiente ``DataFrame``:

In [ ]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

In [ ]:
df.info()

No podemos eliminar valores individuales de un ``DataFrame``; sólo podemos eliminar filas o columnas completas.
Dependiendo de la aplicación, puede que quieras una cosa u otra, por lo que ``dropna()`` da una serie de opciones para un ``DataFrame``.

Por defecto, ``dropna()`` eliminará todas las filas en las que *cualquier* valor nulo esté presente:

In [ ]:
df.dropna()

Alternativamente, puede eliminar los valores nulos a lo largo de un eje diferente; ``axis=1`` elimina todas las columnas que contienen un valor nulo:

In [ ]:
df.dropna(axis=1)
# df.dropna(axis='columns')

Pero esto también elimina algunos datos buenos; quizá le interese más eliminar filas o columnas con *todos* los valores NA, o con una mayoría de valores NA.
Esto se puede especificar mediante los parámetros ``how`` o ``thresh``, que permiten un control preciso del número de nulos que se permiten.

El valor predeterminado es ``how='any'``, de forma que se descartará cualquier fila o columna (dependiendo de la palabra clave ``axis``) que contenga un valor nulo.
También puede especificar ``how='all'``, que sólo eliminará las filas/columnas que sean *todos* valores nulos:

In [ ]:
df[3] = np.nan
df

In [ ]:
df.dropna(axis='columns', how='all')

Para un control más preciso, el parámetro ``thresh`` permite especificar un número mínimo de valores no nulos para la fila/columna que se desea conservar:

In [ ]:
df

In [ ]:
df.dropna(axis='rows', thresh=3)

In [ ]:
df.dropna(axis='rows', thresh=len(df.columns)*0.75)

In [ ]:
df.dropna(axis='columns', thresh=len(df)*0.75)

Aquí se han eliminado la primera y la última fila, porque sólo contienen dos valores no nulos.

### Rellenar valores nulos

A veces, en lugar de eliminar los valores nulos, prefiere sustituirlos por un valor válido.
Este valor podría ser un número único como cero, o podría ser algún tipo de imputación o interpolación a partir de los valores buenos.
Esto se puede hacer in situ usando el método ``isnull()`` como máscara, pero como es una operación tan común Pandas proporciona el método ``fillna()``, que devuelve una copia del array con los valores nulos reemplazados.

Considera la siguiente ``Serie``:

In [ ]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

Podemos rellenar las entradas NA con un único valor, como cero:

In [ ]:
data.fillna(0)

Podemos especificar un forward-fill para propagar el valor anterior hacia adelante:

In [ ]:
# forward-fill
data.ffill()

O podemos especificar un back-fill para propagar los siguientes valores hacia atrás:

In [ ]:
# back-fill
data.bfill()

Para ``DataFrame``s, las opciones son similares, pero también podemos especificar un ``axis`` a lo largo del cual se producen los rellenos:

In [ ]:
df

In [ ]:
df[1]

In [ ]:
df.fillna(df[1].mean())

In [ ]:
df

In [ ]:
df.fillna(method='bfill', axis=1)

Tenga en cuenta que si un valor anterior no está disponible durante un llenado hacia adelante, el valor NA permanece.